In [1]:
from typing import List

import numpy as np

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.state import State, get_z0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.standard_qtomography_estimator import StandardQTomographyEstimationResult

In [2]:
def calc_mse(xs: List[np.array], ys: List[np.array]) -> np.float64:
    points = []
    for x, y in zip(xs, ys):
        point = np.dot(x - y, x - y)
        points.append(point)

    mse = np.mean(points, dtype=np.float64)
    return mse

def convert_to_series(results: List[StandardQTomographyEstimationResult]):
    # calc mse
    var_tmp = [result.estimated_var_sequence for result in results]
    var_tmp = [list(var) for var in zip(*var_tmp)]
    mses = [
        calc_mse(var, true_object.vec) for var in var_tmp
    ]

    # convert to computation time series
    comp_time_tmp = [result.computation_times for result in results]
    comp_time = [list(comp_time) for comp_time in zip(*comp_time_tmp)]

    return mses, comp_time

def calc_estimate(name: str, true_object: State, num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    qst = StandardQst(povms, on_para_eq_constraint=False)    

    # generate empi dists and calc estimate
    results = []
    for iteration in range(iterations):
        seeds = [iteration] * len(num_data)
        empi_dists_seq = qst.generate_empi_dists_sequence(
            true_object, num_data, seeds
        )

        estimator = LinearEstimator()
        reult = estimator.calc_estimate_sequence(qst, empi_dists_seq, is_computation_time_required=True)

        info = {
            "iteration": iteration + 1,
            "data": empi_dists_seq,
            "estimated_var_sequence": reult.estimated_var_sequence,
            "computation_times": reult.computation_times,
        }
        print(info)
        results.append(reult)

    return convert_to_series(results)

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def show_mse(num_data: List[int], mses: List[float]):
    trace = go.Scatter(x=num_data, y=mses, mode='lines+markers')
    data = [trace]
    layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
    fig = go.Figure(data=data, layout=layout)
    fig.show()

def show_computation_times(num_data: List[int], computation_times_sequence: List[List[float]]):
    subplot_titles = [f"number of data = {num}" for num in num_data]
    #fig = make_subplots(rows=1, cols=2, subplot_titles=["number of data = 10", "number of data = 100"])
    fig = make_subplots(rows=1, cols=len(num_data), subplot_titles=subplot_titles)

    for index, computation_times in enumerate(computation_times_sequence):
        trace = go.Histogram(x=computation_times, xbins=dict(start=0, end=1, size=0.1), marker=dict(color="Blue"))
        fig.append_trace(trace, 1, index + 1)

    fig.update_layout(
        title_text="computation times for each estimate",
        xaxis_title_text="computation time(sec)", 
        yaxis_title_text="count", 
        bargap=0.2,
        bargroupgap=0.1,
        showlegend=False,
        width=1200,
    )
    fig.show()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

# settings
true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 10

# calculate
param_affine_est_linear, time_affine_est_linear = calc_estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear, time_linear_est_linear = calc_estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

{'iteration': 1, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([1., 0.]))], [(10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([1., 0.]))], [(100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([1., 0.]))]], 'estimated_var_sequence': [array([0.70710678, 0.01414214, 0.01414214, 0.70710678]), array([0.70710678, 0.02404163, 0.02404163, 0.70710678]), array([0.70710678, 0.00905097, 0.00905097, 0.70710678]), array([0.70710678, 0.00077782, 0.00077782, 0.70710678])], 'computation_times': [0.0, 0.0, 0.0, 0.0]}
{'iteration': 2, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.494, 0.506])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))], [(10000, array([0.499, 0.501])), (10000, array([0.499, 0.501])), (10000, array([1., 0.]))], [(100000, array

In [5]:
show_mse(num_data, param_affine_est_linear)
show_computation_times(num_data, time_affine_est_linear)